In [2]:
%%time
%pip install catboost
#%pip install optuna
#%pip install optuna_distributed
#%pip install openfe
%pip install seaborn
%pip install xgboost
%pip install lightgbm
#%pip install fastkaggle
#%pip install h2o
%pip install -Uqq fastbook
%pip install polars
%pip install tqdm
#%pip install wandb
#%pip install sweetviz
#%pip install --upgrade scipy
#%pip install -q -U autogluon.tabular
#%pip install autogluon


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 203.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 244.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 MB 163.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 106.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 204.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
CPU times: user 615 ms, sys: 123 ms, total: 738 ms
Wall time: 43.6 s


In [4]:
import numpy as np
from numpy import random
#import fastbook
#fastbook.setup_book()
#from fastbook import *
from fastai.tabular.all import *
import pandas as pd
#import matplotlib.pyplot as plt
#from fastai.imports import *
#np.set_printoptions(linewidth=130)
from pathlib import Path
import os
import xgboost as xgb
#from xgboost import plot_importance
from xgboost import XGBRegressor
import warnings
import gc
import pickle
from joblib import dump, load
import typing as t
#import bentoml
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import KFold, cross_val_score,train_test_split
#import wandb


torch.manual_seed(42)



In [5]:
path = Path('data/')

train_df = pd.read_csv(path/'train.csv')
test_df = pd.read_csv(path/'test.csv')
sub_df = pd.read_csv(path/'sample_submission.csv')

In [6]:
missing_values_count = train_df.isnull().sum()
sorted_missing_values = missing_values_count.sort_values(ascending=False)
print(sorted_missing_values)

date     0
store    0
item     0
sales    0
dtype: int64


In [7]:
cont_names,cat_names = cont_cat_split(train_df, dep_var='sales')
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df))
to = TabularPandas(train_df, procs=[Categorify, FillMissing,Normalize],
#to = TabularPandas(train_df, procs=[Categorify,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='sales',
                   y_block=CategoryBlock(),
                   splits=splits)

In [8]:
dls = to.dataloaders(bs=64)
#dls = to.dataloaders(bs=1024)
test_dl = dls.test_dl(test_df)

X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

xgb_model = xgb.XGBRegressor()
xgb_model = xgb_model.fit(X_train, y_train)

#xgb_preds = tensor(xgb_model.predict(test_dl.xs))

#xgb_preds_x = tensor(xgb_model.predict(X_test))

#xgb_score = mean_absolute_percentage_error(y_test,xgb_preds_x)
#print(f"Model accuracy: {xgb_score}")

#bentoml.xgboost.save_model("store_forecast_v1", xgb_model)

In [9]:
xgb_preds = tensor(xgb_model.predict(test_dl.xs))

In [12]:
xgb_preds

tensor([10.8678, 10.8678, 10.8678,  ..., 41.3188, 41.3188, 41.3188])

In [13]:
# Assuming xgb_preds is a PyTorch tensor
first_20_values = xgb_preds[:20]
print(first_20_values)


tensor([10.8678, 10.8678, 10.8678, 10.8678, 10.8678, 10.8678, 10.8678, 10.8678,
        10.8678, 10.8678, 10.8678, 10.8678, 10.8678, 10.8678, 10.8678, 10.8678,
        10.8678, 10.8678, 10.8678, 10.8678])
